In [ ]:
!pip uninstall -y transformers

Found existing installation: transformers 4.35.2
Uninstalling transformers-4.35.2:
  Successfully uninstalled transformers-4.35.2


In [ ]:
!pip install git+https://github.com/huggingface/transformers

  Cloning https://github.com/huggingface/transformers to /tmp/pip-req-build-95179z7q
  Running command git clone --filter=blob:none --quiet https://github.com/huggingface/transformers /tmp/pip-req-build-95179z7q
  Resolved https://github.com/huggingface/transformers to commit ce4fff0be7f6464d713f7ac3e0bbaafbc6959ae5
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  Created wheel for transformers: filename=transformers-4.38.0.dev0-py3-none-any.whl size=8505457 sha256=55ecc758748e99e7f8b258040e88dfe6780a079979ee04f6fca782c94aecace3
  Stored in directory: /tmp/pip-ephem-wheel-cache-_6r49nsp/wheels/c0/14/d6/6c9a5582d2ac191ec0a483be151a4495fe1eb2a6706ca49f1b
Successfully built transformers


In [ ]:
!pip install huggingface_hub

In [ ]:
!pip install torch torchtext sentencepiece pandas tqdm datasets peft==0.4.0 bitsandbytes==0.40.2 trl==0.4.7 accelerate einops scipy

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 536.6/536.6 kB 11.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 72.9/72.9 kB 13.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 92.5/92.5 MB 8.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 77.4/77.4 kB 11.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 280.0/280.0 kB 23.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.6/44.6 kB 4.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 38.3/38.3 MB 49.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 21.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 25.3 MB/s eta 0:00:00
  Attempting uninstall: pyarrow
    Found existing installation: pyarrow 10.0.1
    Uninstalling pyarrow-10.0.1:
      Successfully uninstalled pyarrow-10.0.1
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed.

# Prepare the Data Set for Fine tuning

In [ ]:
import os
from dataclasses import dataclass, field
from typing import Optional

from datasets import load_dataset
from datasets import load_from_disk
from datasets import Dataset

In [ ]:
wikisum_data = load_dataset("abertsch/rescraped-wikisum-test")

Generating train split:   0%|          | 0/180889 [00:00<?, ? examples/s]

In [ ]:
import pandas as pd
# Convert to DataFrame
df = pd.DataFrame(wikisum_data)

# Display the first few rows of the DataFrame
df.head(2)

,train
0,"{'id': '0.1', 'input': 'The Excellent Dizzy Co..."
1,"{'id': '0.2', 'input': 'A break away!<EOT>Not ..."


In [ ]:
df.loc[0]['train'].keys()

dict_keys(['id', 'input', 'title', 'summary', 'full_article', 'section_boundaries'])

# Transform each row into (summary, Article)

In [ ]:
# Function to transform the row into desired format
def format_row(row):
    summary = row['train']['input']
    full_article = row['train']['full_article']
    formatted_string = f"[INST] {summary} [/INST] {full_article} "
    return formatted_string

# Apply the function to each row of the dataframe
df['Formatted'] = df.apply(format_row, axis=1)

# Display the formatted column
df['Formatted']

0         [INST] The Excellent Dizzy Collection<EOT>Earl...
1         [INST] A break away!<EOT>Not what you were loo...
2         [INST] Conasprella kohni<EOT>1 [/INST] Conaspr...
3         [INST] St Oswald's Church, Lassington<EOT>1 [/...
4         [INST] Ori Murray<EOT>Join Kelly in welcoming ...
                                ...                        
180884    [INST] Katharine Jefferts Schori<EOT>Oh my goo...
180885    [INST] Stephanie Kantis<EOT>Jolley's dialogue ...
180886    [INST] Silo High School<EOT>1 [/INST] Silo Hig...
180887    [INST] Board of Canadian Registered Safety Pro...
180888    [INST] Limbo Parks<EOT>We’ve sent a registrati...
Name: Formatted, Length: 180889, dtype: object

In [ ]:
new_df = df.rename(columns={'Formatted': 'Text'})
new_df = new_df[['Text']]

In [ ]:
new_df.head()

,Text
0,[INST] The Excellent Dizzy Collection<EOT>Earl...
1,[INST] A break away!<EOT>Not what you were loo...
2,[INST] Conasprella kohni<EOT>1 [/INST] Conaspr...
3,"[INST] St Oswald's Church, Lassington<EOT>1 [/..."
4,[INST] Ori Murray<EOT>Join Kelly in welcoming ...


In [ ]:
# training_dataset = load_dataset("csv", data_files="formatted_data.csv", split="train")
from datasets import Dataset
training_dataset = Dataset.from_pandas(new_df)

In [ ]:
!huggingface-cli login

In [ ]:
training_dataset.push_to_hub("AbhishekJosyula/Wikisun_QA_Format")

Uploading the dataset shards:   0%|          | 0/2 [00:00<?, ?it/s]

Creating parquet from Arrow format:   0%|          | 0/91 [00:00<?, ?ba/s]

Creating parquet from Arrow format:   0%|          | 0/91 [00:00<?, ?ba/s]

README.md:   0%|          | 0.00/369 [00:00<?, ?B/s]

CommitInfo(commit_url='https://huggingface.co/datasets/AbhishekJosyula/Wikisun_QA_Format/commit/341c332d6417075c0532dba73e939b3b8c53ffae', commit_message='Upload dataset', commit_description='', oid='341c332d6417075c0532dba73e939b3b8c53ffae', pr_url=None, pr_revision=None, pr_num=None)